# AI safety via debate
OAI article: https://openai.com/index/debate/

Paper: https://arxiv.org/pdf/1805.00899

Implementation by Alex Chen

Jul 2024

# Setup

In [13]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from torchvision.transforms import ToTensor
import math

# Training the Judge
The judge is a classifier trained to "predict MNIST digits from 6 non-black digits, sampled at random for each presentation when pretraining."

As per the paper, their original sparse classifier achieved 59.4% accuracy. They use a convolutional neural net.

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.0005
momentum = 0.5
log_interval = 10

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision.transforms import ToTensor

class RandomPixelSelection(object):
    def __init__(self, num_pixels=6):
        self.num_pixels = num_pixels

    def __call__(self, img):
        img_np = np.array(img)
        flattened = img_np.flatten()
        non_black_indices = np.where(flattened > 0)[0]

        if len(non_black_indices) < self.num_pixels:
            selected_indices = non_black_indices
        else:
            selected_indices = np.random.choice(non_black_indices, self.num_pixels, replace=False)

        new_img = np.zeros_like(flattened)
        new_img[selected_indices] = flattened[selected_indices]
        new_img = new_img.reshape(img_np.shape)

        new_img = torch.tensor(new_img, dtype=torch.float32)
        new_img = (new_img - 0.1307) / 0.3081

        return new_img.unsqueeze(0)

transform = torchvision.transforms.Compose([
    RandomPixelSelection(num_pixels=6)
])

batch_size_train = 64
batch_size_test = 1000

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('~/mnist_data/', train=True, download=True, transform=transform),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('~/mnist_data/', train=False, download=True, transform=transform),
    batch_size=batch_size_test, shuffle=True)

class Judge(nn.Module):
    def __init__(self):
        super(Judge, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.fc1 = nn.Linear(12 * 20 * 20, 60)
        self.fc2 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(30, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 12 * 20 * 20)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Judge().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

n_epochs = 8
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.654951
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.112054
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.798163
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.829594
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.591716
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.783678
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.771029
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.430417
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.712763
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.590702

Test set: Average loss: 1.4883, Accuracy: 4791/10000 (48%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.440902
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.430490
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.537496
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.650518
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.371253
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.337301
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.454696
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.538943
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.446250
T

In [ ]:
example_data, example_target = next(iter(test_loader))
example_data, example_target = example_data.to(device), example_target.to(device)
model.eval()
with torch.no_grad():
    output = model(example_data)
    pred = output.argmax(dim=1, keepdim=True)
    for i in range(len(pred)):
        print(f'Predicted: {pred[i].item()}, Actual: {example_target[i].item()}')


In [40]:
class StandardMNIST(object):
    def __call__(self, img):
        img_np = np.array(img)
        new_img = torch.tensor(img_np, dtype=torch.float32)
        return new_img.unsqueeze(0)

standardTransform = torchvision.transforms.Compose([
    StandardMNIST(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [41]:
batch_size_test = 1

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('~/mnist_data/', train=False, download=True, transform=standardTransform),
    batch_size=batch_size_test, shuffle=True)

In [44]:
example_image, example_label = next(iter(test_loader))
example_image_dim = example_image[0][0]
print(example_label)

tensor([6])


# The Agent

OAI paper uses pure Monte Carlo Tree Search with 10k rollouts per move

The selection criteria is the PUCT variant: the win rate + Upper bound

tree layers alternate between the truthful agent and the lying agent

each node can track the:
- parent
- children
- full game state up to that node, which also reveals which agent they are and what turn it is
- value; number of wins


the root of the tree is the empty state

first move is truthful agent

In [233]:
class Node():
    def __init__(self, parent=None, children=None, winRate=None, timesVisited=0, depth=0):
        self.parent = parent
        self.children = children if children is not None else {}
        self.timesVisited = timesVisited
        self.winRate = winRate if winRate is not None else [0,0]
        self.depth = depth

In [296]:
class MCTS():

    def __init__(self, image, targetLabel, model):
        self.root = Node(parent=None, winRate=[0,0], timesVisited=0, depth=0)
        self.visitedLeaves = {} # may not need due to sparsity
        self.image = image
        self.targetLabel = targetLabel
        self.model = model
        self.non_black_pixels = []
        self.winRateWeight = 0.5
        # print(self.image.shape)
        for i in range(self.image.shape[0]):
            for j in range(self.image.shape[1]):
                if self.image[i, j] > -4.23e-01: # -0.42421... is what zeroes get mapped to
                    self.non_black_pixels.append((i, j))
        
        self.numValidActions = len(self.non_black_pixels)
    

    def calculateUtility(self, winRate, sumVisited, numVisits):
        winRateRatio = 0
        if winRate[1] != 0:
            winRateRatio = winRate[0] / winRate[1]
        return self.winRateWeight * winRateRatio + (1 / self.numValidActions) * (math.sqrt(1 + sumVisited)) / (1 + numVisits)


    def chooseNextNode(self, curNode, chosen):
        sumVisited = 0
        for child in curNode.children.values():
            sumVisited += child.timesVisited
        
        maxUtility = -1
        maxChildIndices = []

        for i in range(self.numValidActions):
            if i not in chosen:
                if i in curNode.children:
                    util = self.calculateUtility(curNode.children[i].winRate, sumVisited, curNode.children[i].timesVisited)
                else:
                    util = self.calculateUtility([0,0], sumVisited, 0)

                if util > maxUtility:
                    maxChildIndices = [i]
                    maxUtility = util
                elif util == maxUtility:
                    maxChildIndices.append(i)
        
        maxChildIndex = random.choice(maxChildIndices)
        if maxChildIndex in curNode.children.keys():
            curNode.children[maxChildIndex].timesVisited += 1
            # print(f"Child {maxChildIndex} visited {curNode.children[maxChildIndex].timesVisited} times")
        else:
            curNode.children[maxChildIndex] = Node(parent=curNode, winRate=[0,0], timesVisited=1, depth=curNode.depth+1)

        return curNode.children[maxChildIndex], maxChildIndex


    def backprop(self, leaf, truthWin):
        curNode = leaf
        while curNode.parent is not None:
            if truthWin and curNode.depth % 2 == 1:
                curNode.winRate[0] += 1
            elif not truthWin and curNode.depth % 2 == 0:
                curNode.winRate[0] += 1
            curNode.winRate[1] += 1
            curNode = curNode.parent
    

    def judge(self, chosen):
        judge_image = torch.zeros((28, 28))
        chosen_pixels = [self.non_black_pixels[index] for index in chosen]
        for c in chosen_pixels:
            judge_image[c[0]][c[1]] = self.image[c[0]][c[1]]
        judge_image = judge_image.unsqueeze(0).unsqueeze(0) # this is a batch consisting of a single image lol
        output = self.model(judge_image)
        pred = output.argmax(dim=1, keepdim=True).item()
        return pred == self.targetLabel.item(), pred
    
        
    def rollouts(self, numRollouts):
        for i in range(numRollouts):
            # print("rollout",i)
            curNode = self.root
            chosen = []
            while curNode.depth != 6:
                nextNode, chosenIndex = self.chooseNextNode(curNode, chosen)
                # print(f"Current depth: {curNode.depth}, Chosen index: {chosenIndex}")
                chosen.append(chosenIndex)
                curNode = nextNode
            
            truthWin, pred = self.judge(chosen)
            self.backprop(curNode, truthWin)
    

    def getBestMoves(self):
        curNode = self.root
        moves = []
        while curNode.depth != 6:
            maxVisits = -1
            maxVisitsIndex = -1
            for c in curNode.children.keys():
                if curNode.children[c].timesVisited > maxVisits:
                    maxVisits = curNode.children[c].timesVisited
                    maxVisitsIndex = c
            moves.append(maxVisitsIndex)

            # print(curNode.children[maxVisitsIndex].winRate)
            # print(len(curNode.children[maxVisitsIndex].children.keys()))

            curNode = curNode.children[maxVisitsIndex]
        return moves

In [ ]:

a = MCTS(image=example_image_dim, targetLabel=example_label, model=model)
a.rollouts(10000)
a.getBestMoves()

In [259]:
a.judge([8, 69, 7, 9, 51, 27])

False

In [106]:
print(example_image_dim.shape)
a = MCTS(image=example_image_dim, targetLabel=example_label, model=model)
print(example_label)
chosen = [10,1,3,75,4,155]
print(a.judge(chosen))

torch.Size([28, 28])
torch.Size([28, 28])
tensor([6])
True


# Putting it Together


In [308]:
def runBatchDebates(numDebates, numRollouts):
    final_loader = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('~/mnist_data/', train=False, download=True, transform=standardTransform),
        batch_size=1, shuffle=True)

    countCorrect = 0
    for i in range(numDebates):
        example_image, example_label = next(iter(final_loader))
        example_image_dim = example_image[0][0]

        debate = MCTS(image=example_image_dim, targetLabel=example_label, model=model)
        debate.rollouts(numRollouts)
        bestMoves = debate.getBestMoves()

        correct, pred = debate.judge(bestMoves)
        if correct:
            countCorrect += 1

        print(f"Debate {i+1} | True label: {int(example_label)} | Predicted label: {pred} | Correct ratio: {countCorrect} / {i+1}")

In [309]:
runBatchDebates(100, 10000)

Debate 1 | True label: 7 | Predicted label: 9 | Correct ratio: 0 / 1
Debate 2 | True label: 9 | Predicted label: 9 | Correct ratio: 1 / 2
Debate 3 | True label: 9 | Predicted label: 1 | Correct ratio: 1 / 3
Debate 4 | True label: 1 | Predicted label: 1 | Correct ratio: 2 / 4
Debate 5 | True label: 4 | Predicted label: 9 | Correct ratio: 2 / 5
Debate 6 | True label: 1 | Predicted label: 3 | Correct ratio: 2 / 6
Debate 7 | True label: 5 | Predicted label: 2 | Correct ratio: 2 / 7
Debate 8 | True label: 2 | Predicted label: 2 | Correct ratio: 3 / 8
Debate 9 | True label: 8 | Predicted label: 9 | Correct ratio: 3 / 9
Debate 10 | True label: 6 | Predicted label: 0 | Correct ratio: 3 / 10
Debate 11 | True label: 6 | Predicted label: 6 | Correct ratio: 4 / 11
Debate 12 | True label: 2 | Predicted label: 2 | Correct ratio: 5 / 12
Debate 13 | True label: 9 | Predicted label: 7 | Correct ratio: 5 / 13
Debate 14 | True label: 8 | Predicted label: 8 | Correct ratio: 6 / 14
Debate 15 | True label: 